# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298687897248                   -0.85    4.1
  2   -8.300234473025       -2.81       -1.25    1.0
  3   -8.300451792609       -3.66       -1.89    3.4
  4   -8.300463326560       -4.94       -2.78    1.2
  5   -8.300464530709       -5.92       -3.17    7.2
  6   -8.300464601668       -7.15       -3.31    1.8
  7   -8.300464621939       -7.69       -3.44    2.1
  8   -8.300464633806       -7.93       -3.58    1.0
  9   -8.300464640525       -8.17       -3.75    1.5
 10   -8.300464642038       -8.82       -3.85    6.2
 11   -8.300464643598       -8.81       -4.10    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64179847221                   -0.70    5.6
  2   -16.67860194964       -1.43       -1.14    2.0
  3   -16.67923787324       -3.20       -1.87    3.1
  4   -16.67928045809       -4.37       -2.76    2.4
  5   -16.67928603755       -5.25       -3.14    5.6
  6   -16.67928621814       -6.74       -3.52    1.6
  7   -16.67928622233       -8.38       -4.03    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32427536905                   -0.56    6.4
  2   -33.33260630785       -2.08       -0.99    1.0
  3   -33.33587366903       -2.49       -1.68    3.6
  4   -33.33613165517       -3.59       -2.60    3.6
  5   -33.33681038896       -3.17       -2.24    9.0
  6   -33.33682205249       -4.93       -2.27    1.0
  7   -33.33694297573       -3.92       -3.22    3.2
  8   -33.33694151011   +   -5.83       -3.11    7.9
  9   -33.33694389744       -5.62       -3.93    4.5
 10   -33.33694391028       -7.89       -4.09    4.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298607211046                   -0.85    4.1
  2   -8.300285166228       -2.78       -1.59    1.5
  3   -8.300442579944       -3.80       -2.50    8.8
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│      21
│       5
│       4
│       4
│       4
│       4
│       4
│     100
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:36
  4   -8.300351465593   +   -4.04       -2.23   18.2
  5   -8.300464454712       -3.95       -3.64    3.6
  6   -8.300464586092       -6.88       -3.91    3.8
  7   -8.300464636001       -7.30       -4.32    2.6


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32562847731                   -0.56    6.6
  2   -33.30898644981   +   -1.78       -1.26    1.0
  3   -10.26777718699   +    1.36       -0.38    7.9
  4   -33.31917344969        1.36       -1.61    7.1
  5   -33.25595721061   +   -1.20       -1.36    3.6
  6   -32.52970115248   +   -0.14       -1.11    4.5
  7   -33.32587414188       -0.10       -2.04    6.6
  8   -33.33672916574       -1.96       -2.48    2.9
  9   -33.33666355425   +   -4.18       -2.52    3.4
 10   -33.33687913763       -3.67       -2.95    2.0
 11   -33.33691671652       -4.43       -3.13    1.9
 12   -33.33694274071       -4.58       -3.72    2.9
 13   -33.33694380529       -5.97       -4.00    4.2


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.